<a href="https://colab.research.google.com/github/Prince-74/30days-DSA-Challenge/blob/main/final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Run this first cell (may take 1-2 mins)
!pip install -q transformers datasets seqeval accelerate peft==0.7.0 evaluate


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.3/168.3 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 6.2 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')
# Put your two CSVs in: /content/drive/MyDrive/ai_med_data/


Mounted at /content/drive


In [3]:
import os, json, random, math, csv, io
from pathlib import Path

DATA_DIR = "/content/drive/MyDrive/ai_med_data"  # change if needed
OUT_DIR = "/content/ai_med_output"
os.makedirs(OUT_DIR, exist_ok=True)

MODEL_NAME = "emilyalsentzer/Bio_ClinicalBERT"
MAX_LEN = 256
RANDOM_SEED = 42
random.seed(RANDOM_SEED)


In [4]:
import pandas as pd

med_path = os.path.join(DATA_DIR, "medicines.csv")
dis_path = os.path.join(DATA_DIR, "Diseases_and_Symptoms.csv")

med_df = pd.read_csv(med_path)
dis_df = pd.read_csv(dis_path, header=None)  # your disease file looked strange — inspect
print("med_df rows:", len(med_df))
print("disease_df shape:", dis_df.shape)
display(med_df.head())
display(dis_df.head(3))


/tmp/ipython-input-3544059305.py:7: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,2

med_df rows: 253973
disease_df shape: (246946, 378)


,id,name,price(₹),Is_discontinued,manufacturer_name,type,pack_size_label,short_composition1,short_composition2
0,1,Augmentin 625 Duo Tablet,223.42,False,Glaxo SmithKline Pharmaceuticals Ltd,allopathy,strip of 10 tablets,Amoxycillin (500mg),Clavulanic Acid (125mg)
1,2,Azithral 500 Tablet,132.36,False,Alembic Pharmaceuticals Ltd,allopathy,strip of 5 tablets,Azithromycin (500mg),NaN
2,3,Ascoril LS Syrup,118.00,False,Glenmark Pharmaceuticals Ltd,allopathy,bottle of 100 ml Syrup,Ambroxol (30mg/5ml),Levosalbutamol (1mg/5ml)
3,4,Allegra 120mg Tablet,218.81,False,Sanofi India Ltd,allopathy,strip of 10 tablets,Fexofenadine (120mg),NaN
4,5,Avil 25 Tablet,10.96,False,Sanofi India Ltd,allopathy,strip of 15 tablets,Pheniramine (25mg),NaN


,0,1,2,3,4,5,6,7,8,9,...,368,369,370,371,372,373,374,375,376,377
0,diseases,anxiety and nervousness,depression,shortness of breath,depressive or psychotic symptoms,sharp chest pain,dizziness,insomnia,abnormal involuntary movements,chest tightness,...,stuttering or stammering,problems with orgasm,nose deformity,lump over jaw,sore in nose,hip weakness,back swelling,ankle stiffness or tightness,ankle weakness,neck weakness
1,panic disorder,1,0,1,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,panic disorder,0,0,1,1,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
import re
from tqdm.auto import tqdm

# Extract med names (simple heuristics)
def extract_med_names(med_df):
    if "name" in med_df.columns:
        return med_df["name"].astype(str).tolist()
    # fallback look for second column
    return med_df.iloc[:,1].astype(str).tolist()

med_names = extract_med_names(med_df)
print("sample meds:", med_names[:5])

# Disease parsing: if file is single-column with disease names in first col
if dis_df.shape[1] == 1:
    disease_list = dis_df.iloc[:,0].astype(str).tolist()
else:
    # if first row is header of symptom names, then later rows have disease name + flags
    # try to infer structure:
    if dis_df.shape[0] > 1 and dis_df.iloc[0,0].lower().strip() == "diseases":
        # assume proper CSV with header
        dis_df2 = pd.read_csv(dis_path)
        disease_list = dis_df2['diseases'].tolist()
    else:
        # fallback: take unique values across first column
        disease_list = dis_df.iloc[:,0].astype(str).unique().tolist()

print("sample diseases:", disease_list[:10])

# Simple templates
templates = [
    "Patient reports {symptoms}. Likely condition: {disease}. Current medicines: {meds}.",
    "{disease} patient with symptoms of {symptoms}. Medicines being taken: {meds}.",
    "Symptoms: {symptoms}. Diagnosis: {disease}. Medications: {meds}.",
]

def escape_for_text(s):
    return s.replace("\n"," ").strip()

# generate examples
examples = []
N_PER_DISEASE = 4  # increase if you want more synthetic data
for disease in disease_list:
    disease = escape_for_text(disease)
    if not disease or disease.lower().strip() in ["nan","none"]:
        continue
    for _ in range(N_PER_DISEASE):
        # pick 2-4 random symptoms and 1-2 meds
        symptoms = random.sample([
            "fever","cough","shortness of breath","headache","nausea","vomiting",
            "dizziness","pain","fatigue","chills","sore throat","runny nose",
        ], k=random.randint(1,3))
        meds = random.sample(med_names, k=min(2, max(1, len(med_names)//20 or 1)))
        # build text
        tmpl = random.choice(templates)
        symptom_text = ", ".join(symptoms)
        med_text = ", ".join(meds)
        text = tmpl.format(symptoms=symptom_text, disease=disease, meds=med_text)
        text = escape_for_text(text)

        # entities: find spans (naive substring search; ensure it matches)
        entities = []
        # add disease entity (first match)
        idx = text.lower().find(disease.lower())
        if idx != -1:
            entities.append({"start": idx, "end": idx + len(disease), "label": "DISEASE"})
        # add symptom spans
        for s in symptoms:
            idx2 = text.lower().find(s.lower())
            if idx2 != -1:
                entities.append({"start": idx2, "end": idx2 + len(s), "label": "SYMPTOM"})
        # add med spans
        for m in meds:
            idx3 = text.lower().find(m.lower())
            if idx3 != -1:
                entities.append({"start": idx3, "end": idx3 + len(m), "label": "DRUG"})

        examples.append({"text": text, "entities": entities})

print("Synthetic examples generated:", len(examples))

# Save to JSONL
out_jsonl = os.path.join(OUT_DIR, "train_synthetic.jsonl")
with open(out_jsonl, "w", encoding="utf-8") as f:
    for ex in examples:
        f.write(json.dumps(ex, ensure_ascii=False) + "\n")
print("Saved synthetic JSONL ->", out_jsonl)


sample meds: ['Augmentin 625 Duo Tablet', 'Azithral 500 Tablet', 'Ascoril LS Syrup', 'Allegra 120mg Tablet', 'Avil 25 Tablet']
sample diseases: ['panic disorder', 'panic disorder', 'panic disorder', 'panic disorder', 'panic disorder', 'panic disorder', 'panic disorder', 'panic disorder', 'panic disorder', 'panic disorder']
Synthetic examples generated: 987780
Saved synthetic JSONL -> /content/ai_med_output/train_synthetic.jsonl


In [6]:
# quick validation: every line must be valid JSON and have text & entities
clean_path = os.path.join(OUT_DIR, "train_clean_final.jsonl")
good = 0
bad = 0

with open(out_jsonl, "r", encoding="utf-8") as fr, open(clean_path, "w", encoding="utf-8") as fw:
    for i, line in enumerate(fr):
        try:
            obj = json.loads(line)
            if "text" in obj and "entities" in obj:
                fw.write(json.dumps(obj, ensure_ascii=False) + "\n")
                good += 1
            else:
                bad += 1
        except Exception as e:
            bad += 1

print("cleaned lines:", good, "bad lines removed:", bad)
print("cleaned JSONL saved to:", clean_path)


cleaned lines: 987780 bad lines removed: 0
cleaned JSONL saved to: /content/ai_med_output/train_clean_final.jsonl


In [7]:
from datasets import load_dataset
from transformers import AutoTokenizer

# ----------------------------
# LOAD TOKENIZER + DATASET
# ----------------------------
JSONL_PATH = clean_path   # your cleaned JSONL path
MODEL_NAME = "emilyalsentzer/Bio_ClinicalBERT"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
ds = load_dataset("json", data_files=JSONL_PATH, split="train")
print("Loaded dataset rows:", len(ds))

# ----------------------------
# LABELS
# ----------------------------
labels_list = ["O","B-DISEASE","I-DISEASE","B-SYMPTOM","I-SYMPTOM","B-DRUG","I-DRUG"]
label2id = {l:i for i,l in enumerate(labels_list)}
id2label = {i:l for i,l in enumerate(labels_list)}

MAX_LEN = 256

# ----------------------------
# BIO TAG GENERATOR
# ----------------------------
def get_bio_tags(text, entities, offsets):
    tags = ["O"] * len(offsets)

    for ent in entities:
        s, e = ent["start"], ent["end"]
        lab = ent["label"]
        start_flag = True

        for i, (tok_s, tok_e) in enumerate(offsets):
            if tok_s is None:
                continue
            if tok_s >= s and tok_e <= e:
                tags[i] = ("B-" + lab) if start_flag else ("I-" + lab)
                start_flag = False

    return [label2id.get(t, label2id["O"]) for t in tags]

# ----------------------------
# TOKENIZE BATCH
# ----------------------------
def tokenize_batch(batch):
    tok = tokenizer(
        batch["text"],
        truncation=True,
        padding="max_length",
        max_length=MAX_LEN,
        return_offsets_mapping=True
    )

    all_labels = []

    for i in range(len(batch["text"])):
        offsets = tok["offset_mapping"][i]
        offsets_clean = [(None, None) if (s == 0 and e == 0) else (s, e) for (s, e) in offsets]

        labels = get_bio_tags(batch["text"][i], batch["entities"][i], offsets_clean)
        labels = labels[:MAX_LEN] + [label2id["O"]] * (MAX_LEN - len(labels))

        all_labels.append(labels)

    tok["labels"] = all_labels
    tok.pop("offset_mapping")
    return tok

print("Tokenizing dataset... this will take time")
encoded_dataset = ds.map(tokenize_batch, batched=True, batch_size=200)
print("Tokenization Complete")

# ----------------------------
# REDUCE DATASET SIZE (OPTIONAL)
# ----------------------------
encoded_dataset = encoded_dataset.shuffle(seed=42).select(range(50000))

# ----------------------------
# TRAIN / TEST SPLIT
# ----------------------------
train_ds = encoded_dataset.select(range(40000))
eval_ds  = encoded_dataset.select(range(40000, 50000))

# ----------------------------
# TORCH FORMAT
# ----------------------------
train_ds.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
eval_ds.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

print("Final Train size:", len(train_ds))
print("Final Eval size:", len(eval_ds))


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

Loaded dataset rows: 987780
Tokenizing dataset... this will take time


Map:   0%|          | 0/987780 [00:00<?, ? examples/s]

Tokenization Complete
Final Train size: 40000
Final Eval size: 10000


old

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer

JSONL_PATH = clean_path
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
ds = load_dataset("json", data_files=JSONL_PATH, split="train")
print("Loaded dataset rows:", len(ds))

labels_list = ["O","B-DISEASE","I-DISEASE","B-SYMPTOM","I-SYMPTOM","B-DRUG","I-DRUG"]
label2id = {l:i for i,l in enumerate(labels_list)}
id2label = {i:l for i,l in enumerate(labels_list)}

MAX_LEN = 256

def get_bio_tags(text, entities, offsets):
    tags = ["O"] * len(offsets)
    for ent in entities:
        s,e = ent["start"], ent["end"]
        lab = ent["label"]
        first = True
        for i, (tok_s, tok_e) in enumerate(offsets):
            if tok_s is None:
                continue
            if tok_s >= s and tok_e <= e:
                tags[i] = ("B-" if first else "I-") + lab
                first = False
    return [label2id.get(t, label2id["O"]) for t in tags]

def tokenize_batch(batch):
    tok = tokenizer(batch["text"], truncation=True, padding="max_length",
                    max_length=MAX_LEN, return_offsets_mapping=True)
    all_labels = []
    for i in range(len(batch["text"])):
        offsets = tok["offset_mapping"][i]
        # unify offset tuples; convert (0,0) -> (None,None)
        offsets_clean = [(None,None) if (s==0 and e==0) else (s,e) for (s,e) in offsets]
        labels = get_bio_tags(batch["text"][i], batch["entities"][i], offsets_clean)
        labels = labels[:MAX_LEN] + [label2id["O"]] * (MAX_LEN - len(labels))
        all_labels.append(labels)
    tok["labels"] = all_labels
    tok.pop("offset_mapping")
    return tok

print("Tokenizing (this may take several minutes)...")
encoded = ds.map(tokenize_batch, batched=True, batch_size=200)
print(encoded)


In [ ]:
from datasets import DatasetDict
dataset = encoded.train_test_split(test_size=0.1, seed=RANDOM_SEED)
train_ds = dataset["train"]
eval_ds = dataset["test"]

# set torch format
train_ds.set_format(type="torch", columns=["input_ids","attention_mask","labels"])
eval_ds.set_format(type="torch", columns=["input_ids","attention_mask","labels"])

print("Train size:", len(train_ds), " Eval size:", len(eval_ds))


In [9]:
import os
os.environ["WANDB_DISABLED"] = "true"


In [ ]:
# Tokenize dataset
encoded_dataset = ds.map(tokenize_batch, batched=True, batch_size=200)

# ----------------------------
# REDUCE DATASET SIZE (50,000)
# ----------------------------
encoded_dataset = encoded_dataset.shuffle(seed=42).select(range(50000))

# ----------------------------
# SPLIT INTO TRAIN / EVAL
# ----------------------------
train_ds = encoded_dataset.select(range(40000))
eval_ds  = encoded_dataset.select(range(40000, 50000))


In [10]:
from transformers import AutoModelForTokenClassification, DataCollatorForTokenClassification, TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model

num_labels = len(labels_list)

# load base model with token classification head
model = AutoModelForTokenClassification.from_pretrained(
    MODEL_NAME,
    num_labels=num_labels,
    id2label=id2label,
    label2id=label2id
)

# Freeze base model parameters
for p in model.parameters():
    p.requires_grad = False

# apply LoRA
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["query","key","value"],
    lora_dropout=0.05,
    bias="none",
    task_type="TOKEN_CLS",
)
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

data_collator = DataCollatorForTokenClassification(tokenizer)

training_args = TrainingArguments(
    output_dir="/content/ai_med_lora",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-4,

    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    gradient_accumulation_steps=2,

    num_train_epochs=3,
    weight_decay=0.01,
    logging_steps=200,
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    fp16=True
)


# compute_metrics using seqeval
import numpy as np
from seqeval.metrics import precision_score, recall_score, f1_score, classification_report

label_map = {i: lbl for i,lbl in enumerate(labels_list)}

def align_preds_and_labels(preds, labels):
    preds = np.argmax(preds, axis=2)
    batch_preds = []
    batch_labels = []
    for p_seq, l_seq in zip(preds, labels):
        seq_preds = []
        seq_labels = []
        for p,l in zip(p_seq, l_seq):
            seq_preds.append(label_map[int(p)])
            seq_labels.append(label_map[int(l)])
        batch_preds.append(seq_preds)
        batch_labels.append(seq_labels)
    return batch_preds, batch_labels

def compute_metrics(p):
    preds, labels = p
    pred_list, label_list = align_preds_and_labels(preds, labels)
    # seqeval expects label sequences without special tokens; our labels include O padded tokens.
    # It's OK — seqeval will handle O tags.
    p = precision_score(label_list, pred_list)
    r = recall_score(label_list, pred_list)
    f1 = f1_score(label_list, pred_list)
    return {"precision": p, "recall": r, "f1": f1}

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=eval_ds,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

# Run training
trainer.train()

Some weights of BertForTokenClassification were not initialized from the model checkpoint at emilyalsentzer/Bio_ClinicalBERT and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
/tmp/ipython-input-2749001810.py:82: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


trainable params: 447,751 || all params: 108,172,814 || trainable%: 0.4139219305138905


Epoch,Training Loss,Validation Loss,Precision,Recall,F1
1,0.000300,0.000155,0.999298,0.999178,0.999238
2,0.000200,0.000125,0.999118,0.999078,0.999098
3,0.000100,0.000102,0.999339,0.999198,0.999268


TrainOutput(global_step=3750, training_loss=0.012852180182933808, metrics={'train_runtime': 1862.0432, 'train_samples_per_second': 64.445, 'train_steps_per_second': 2.014, 'total_flos': 1.576104357888e+16, 'train_loss': 0.012852180182933808, 'epoch': 3.0})

In [11]:
save_dir = "/content/drive/MyDrive/ai_med_output/lora_model"
os.makedirs(save_dir, exist_ok=True)
model.save_pretrained(save_dir)
tokenizer.save_pretrained(save_dir)
print("Saved to:", save_dir)


Saved to: /content/drive/MyDrive/ai_med_output/lora_model


In [12]:
from peft import PeftModel
from transformers import AutoModelForTokenClassification

base = AutoModelForTokenClassification.from_pretrained(MODEL_NAME, num_labels=num_labels)
model = PeftModel.from_pretrained(base, save_dir)


Some weights of BertForTokenClassification were not initialized from the model checkpoint at emilyalsentzer/Bio_ClinicalBERT and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
from peft import PeftModel
ner_model = PeftModel.from_pretrained(base, save_dir)
ner_model = ner_model.merge_and_unload()   # <-- ADD THIS


In [13]:
from transformers import pipeline
import torch

device = 0 if torch.cuda.is_available() else -1

# load pipeline using saved model (PEFT)
from peft import PeftModel
base = AutoModelForTokenClassification.from_pretrained(MODEL_NAME, num_labels=num_labels)
ner_model = PeftModel.from_pretrained(base, save_dir)
ner_model.to("cuda" if torch.cuda.is_available() else "cpu")

nlp = pipeline("token-classification", model=ner_model, tokenizer=tokenizer, device=device, aggregation_strategy="simple")

def extract_entities(text):
    res = nlp(text)
    # res items: {'entity_group':'DRUG','score':0.9,'word':'Azithral 500 Tablet','start':..., 'end':...}
    return res

# test
print(extract_entities("Patient has fever and cough. Taking Azithral 500 Tablet and Augmentin 625 Duo Tablet."))


Some weights of BertForTokenClassification were not initialized from the model checkpoint at emilyalsentzer/Bio_ClinicalBERT and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Device set to use cuda:0
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


[{'entity_group': 'LABEL_0', 'score': np.float32(0.9998375), 'word': 'patient has', 'start': 0, 'end': 11}, {'entity_group': 'LABEL_3', 'score': np.float32(0.99975544), 'word': 'fever', 'start': 12, 'end': 17}, {'entity_group': 'LABEL_4', 'score': np.float32(0.9513493), 'word': 'and', 'start': 18, 'end': 21}, {'entity_group': 'LABEL_3', 'score': np.float32(0.99532557), 'word': 'cough', 'start': 22, 'end': 27}, {'entity_group': 'LABEL_0', 'score': np.float32(0.99994326), 'word': '. taking', 'start': 27, 'end': 35}, {'entity_group': 'LABEL_5', 'score': np.float32(0.9997825), 'word': 'a', 'start': 36, 'end': 37}, {'entity_group': 'LABEL_6', 'score': np.float32(0.99963206), 'word': '##zithral 500 tablet and', 'start': 37, 'end': 59}, {'entity_group': 'LABEL_5', 'score': np.float32(0.9952034), 'word': 'au', 'start': 60, 'end': 62}, {'entity_group': 'LABEL_6', 'score': np.float32(0.9999604), 'word': '##gmentin 625 duo tablet', 'start': 62, 'end': 84}, {'entity_group': 'LABEL_0', 'score': np.

In [15]:
from transformers import AutoModelForTokenClassification, pipeline
from peft import PeftModel
import torch

device = 0 if torch.cuda.is_available() else -1

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

# Load base model + LoRA
base = AutoModelForTokenClassification.from_pretrained(
    MODEL_NAME, num_labels=num_labels, id2label=id2label, label2id=label2id
)

ner_model = PeftModel.from_pretrained(base, save_dir)
ner_model = ner_model.merge_and_unload()  # IMPORTANT
ner_model.to("cuda" if torch.cuda.is_available() else "cpu")

# Pipeline
nlp = pipeline(
    "token-classification",
    model=ner_model,
    tokenizer=tokenizer,
    device=device,
    aggregation_strategy="simple"
)

def extract_entities(text):
    return nlp(text)

# Test
print(extract_entities(
    "Patient has fever and cough. Taking Azithral 500 Tablet and Augmentin 625 Duo Tablet."
))


Some weights of BertForTokenClassification were not initialized from the model checkpoint at emilyalsentzer/Bio_ClinicalBERT and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Device set to use cuda:0
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


[{'entity_group': 'SYMPTOM', 'score': np.float32(0.9755524), 'word': 'fever and', 'start': 12, 'end': 21}, {'entity_group': 'SYMPTOM', 'score': np.float32(0.99532557), 'word': 'cough', 'start': 22, 'end': 27}, {'entity_group': 'DRUG', 'score': np.float32(0.9996535), 'word': 'azithral 500 tablet and', 'start': 36, 'end': 59}, {'entity_group': 'DRUG', 'score': np.float32(0.9991676), 'word': 'augmentin 625 duo tablet', 'start': 60, 'end': 84}]
